In [1]:
!date

Sun Oct  7 23:32:07 CST 2018


In [2]:
import os
import os.path as osp
import pandas as pd
import numpy as np

In [3]:
import sys

sys.path.insert(0, "../scripts/")

In [4]:
%load_ext autoreload

In [5]:
%aimport etl

In [6]:
%autoreload 1

In [7]:
from etl import preprocess, extract

In [8]:
pd.set_option("display.max_rows", 20)

In [9]:
concept_mapping = {'Country': 'country',
 'Year': 'year',
 'Cause': 'cause',
 'Sex': 'sex',
 'Deaths1': 'deaths_all_ages',
 'Deaths2': 'deaths_age_0_year',
 'Deaths3': 'deaths_age_1_year',
 'Deaths4': 'deaths_age_2_years',
 'Deaths5': 'deaths_age_3_years',
 'Deaths6': 'deaths_age_4_years',
 'Deaths7': 'deaths_age_5_9_years',
 'Deaths8': 'deaths_age_10_14_years',
 'Deaths9': 'deaths_age_15_19_years',
 'Deaths10': 'deaths_age_20_24_years',
 'Deaths11': 'deaths_age_25_29_years',
 'Deaths12': 'deaths_age_30_34_years',
 'Deaths13': 'deaths_age_35_39_years',
 'Deaths14': 'deaths_age_40_44_years',
 'Deaths15': 'deaths_age_45_49_years',
 'Deaths16': 'deaths_age_50_54_years',
 'Deaths17': 'deaths_age_55_59_years',
 'Deaths18': 'deaths_age_60_64_years',
 'Deaths19': 'deaths_age_65_69_years',
 'Deaths20': 'deaths_age_70_74_years',
 'Deaths21': 'deaths_age_75_79_years',
 'Deaths22': 'deaths_age_80_84_years',
 'Deaths23': 'deaths_age_85_89_years',
 'Deaths24': 'deaths_age_90_94_years',
 'Deaths25': 'deaths_age_95_years_and_above',
 'Deaths26': 'deaths_age_unspecified',
 'IM_deaths1': 'infant_deaths_age_0_day',
 'IM_deaths2': 'infant_deaths_age_1_6_days',
 'IM_deaths3': 'infant_deaths_age_7_27_days',
 'IM_deaths4': 'infant_deaths_age_28_364_days',
 'IM_Deaths1': 'infant_deaths_age_0_day',
 'IM_Deaths2': 'infant_deaths_age_1_6_days',
 'IM_Deaths3': 'infant_deaths_age_7_27_days',
 'IM_Deaths4': 'infant_deaths_age_28_364_days'}

In [10]:
d7 = { "A138": "traffic"
    , "A148": "suicide"
    , "A149": "homicide"
    }

In [11]:
d8 = { "A138": "traffic"
    , "A147": "suicide"
    , "A148": "homicide"
    }

In [12]:
d9 = {"B471": "traffic"
      , "B472": "traffic"  # TODO: late effects from traffic is missing (E929.0). here using B529
      , "B529": "traffic"
     , "B54": "suicide"
     , "B55": "homicide"
     }

In [13]:
d10 = {}

In [14]:
# categories of suicide in ICD 10 
suicide = [*[f'X{x}' for x in range(60, 85)],
           *[f'X{x}' for x in range(600, 850)]
          ]
suicide.append("Y870")

# categories of homicide in ICD 10 
homicide = [*[f'X{x}' for x in range(85, 100)],
            *[f'X{x}' for x in range(850, 1000)]
           ]

homicide2 = ['Y00', 'Y01', 'Y02', 'Y03', 'Y04', 'Y05', 
             'Y060', 'Y061', 'Y062', 'Y068', 'Y069',
             'Y070', 'Y071', 'Y072', 'Y073', 'Y078', 'Y079',
             'Y08', 'Y09', 'Y871'
            ]

homicide3 = [*[f'Y00{x}' for x in range(1, 10)],
             *[f'Y0{x}' for x in range(10, 100)],
            ]

for h in homicide2:
    homicide.append(h)
    
for h in homicide3:
    homicide.append(h)

In [15]:
print(suicide)

['X60', 'X61', 'X62', 'X63', 'X64', 'X65', 'X66', 'X67', 'X68', 'X69', 'X70', 'X71', 'X72', 'X73', 'X74', 'X75', 'X76', 'X77', 'X78', 'X79', 'X80', 'X81', 'X82', 'X83', 'X84', 'X600', 'X601', 'X602', 'X603', 'X604', 'X605', 'X606', 'X607', 'X608', 'X609', 'X610', 'X611', 'X612', 'X613', 'X614', 'X615', 'X616', 'X617', 'X618', 'X619', 'X620', 'X621', 'X622', 'X623', 'X624', 'X625', 'X626', 'X627', 'X628', 'X629', 'X630', 'X631', 'X632', 'X633', 'X634', 'X635', 'X636', 'X637', 'X638', 'X639', 'X640', 'X641', 'X642', 'X643', 'X644', 'X645', 'X646', 'X647', 'X648', 'X649', 'X650', 'X651', 'X652', 'X653', 'X654', 'X655', 'X656', 'X657', 'X658', 'X659', 'X660', 'X661', 'X662', 'X663', 'X664', 'X665', 'X666', 'X667', 'X668', 'X669', 'X670', 'X671', 'X672', 'X673', 'X674', 'X675', 'X676', 'X677', 'X678', 'X679', 'X680', 'X681', 'X682', 'X683', 'X684', 'X685', 'X686', 'X687', 'X688', 'X689', 'X690', 'X691', 'X692', 'X693', 'X694', 'X695', 'X696', 'X697', 'X698', 'X699', 'X700', 'X701', 'X702', 

In [16]:
for i in homicide:
    d10[i] = "homicide"

In [17]:
for i in suicide:
    d10[i] = "suicide"

In [18]:
# categories of traffic injures in ICD 10 

traffic = [
    "V011", "V019",
    "V021", "V029",
    "V031", "V039",
    "V041", "V049",
    "V061", "V069",
    "V092", "V093",
    
    "V103", "V104", "V105", "V109",
    "V113", "V114", "V115", "V119",
    "V123", "V124", "V125", "V129",
    "V133", "V134", "V135", "V139",
    "V143", "V144", "V145", "V149",
    "V154", "V155", "V159",
    "V164", "V165", "V169",
    "V174", "V175", "V179",
    "V184", "V185", "V189",
    "V194", "V195", "V196", "V198", "V199",
    
    "V203", "V204", "V205", "V209",
    "V213", "V214", "V215", "V219",
    "V223", "V224", "V225", "V229",
    "V233", "V234", "V235", "V239",
    "V243", "V244", "V245", "V249",
    "V253", "V254", "V255", "V259",
    "V263", "V264", "V265", "V269",
    "V273", "V274", "V275", "V279",
    "V283", "V284", "V285", "V289",
    "V294", "V295", "V296", "V298", "V299",
    
    "V304", "V305", "V306", "V307", "V309",
    "V314", "V315", "V316", "V317", "V319",
    "V324", "V325", "V326", "V327", "V329",
    "V334", "V335", "V336", "V337", "V339",
    "V344", "V345", "V346", "V347", "V349",
    "V354", "V355", "V356", "V357", "V359",
    "V364", "V365", "V366", "V367", "V369",
    "V374", "V375", "V376", "V377", "V379",
    "V384", "V385", "V386", "V387", "V389",
    "V394", "V395", "V396", "V398", "V399",
    
    "V404", "V405", "V406", "V407", "V409",
    "V414", "V415", "V416", "V417", "V419",
    "V424", "V425", "V426", "V427", "V429",
    "V434", "V435", "V436", "V437", "V439",
    "V444", "V445", "V446", "V447", "V449",
    "V454", "V455", "V456", "V457", "V459",
    "V464", "V465", "V466", "V467", "V469",
    "V474", "V475", "V476", "V477", "V479",
    "V484", "V485", "V486", "V487", "V489",
    "V494", "V495", "V496", "V498", "V499",
    
    "V504", "V505", "V506", "V507", "V509",
    "V514", "V515", "V516", "V517", "V519",
    "V524", "V525", "V526", "V527", "V529",
    "V534", "V535", "V536", "V537", "V539",
    "V544", "V545", "V546", "V547", "V549",
    "V554", "V555", "V556", "V557", "V559",
    "V564", "V565", "V566", "V567", "V569",
    "V574", "V575", "V576", "V577", "V579",
    "V584", "V585", "V586", "V587", "V589",
    "V594", "V595", "V596", "V598", "V599",
    
    "V604", "V605", "V606", "V607", "V609",
    "V614", "V615", "V616", "V617", "V619",
    "V624", "V625", "V626", "V627", "V629",
    "V634", "V635", "V636", "V637", "V639",
    "V644", "V645", "V646", "V647", "V649",
    "V654", "V655", "V656", "V657", "V659",
    "V664", "V665", "V666", "V667", "V669",
    "V674", "V675", "V676", "V677", "V679",
    "V684", "V685", "V686", "V687", "V689",
    "V694", "V695", "V696", "V698", "V699",
    
    "V704", "V705", "V706", "V707", "V709",
    "V714", "V715", "V716", "V717", "V719",
    "V724", "V725", "V726", "V727", "V729",
    "V734", "V735", "V736", "V737", "V739",
    "V744", "V745", "V746", "V747", "V749",
    "V754", "V755", "V756", "V757", "V759",
    "V764", "V765", "V766", "V767", "V769",
    "V774", "V775", "V776", "V777", "V779",
    "V784", "V785", "V786", "V787", "V789",
    "V794", "V795", "V796", "V798", "V799",
    
    "V803", "V804", "V805",
    "V811",
    "V821", "V828", "V829",
    "V830", "V831", "V832", "V833",
    "V840", "V841", "V842", "V843",
    "V850", "V851", "V852", "V853",
    "V860", "V861", "V862", "V863",
    "V870", "V871", "V872", "V873", "V874", "V875", "V876", "V877","V878", "V879",

    "V892", "V893",
    "V899",
    "V99",
    "Y850"
]

for i in traffic:
    d10[i] = "traffic"

In [19]:
icd7 = pd.read_csv('../source/morticd07.zip')

/Users/semio/.pyenv/versions/3.6.5/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [20]:
df1 = preprocess(icd7, d7.keys())

In [21]:
df1.Cause.unique()

array(['A138', 'A148', 'A149'], dtype=object)

In [22]:
df1

,Country,Year,Cause,Sex,Deaths1,Deaths2,Deaths3,Deaths4,Deaths5,Deaths6,...,Deaths21,Deaths22,Deaths23,Deaths24,Deaths25,Deaths26,IM_Deaths1,IM_Deaths2,IM_Deaths3,IM_Deaths4
266,1125,1955,A138,1,66,1,0,0.0,3.0,2.0,...,0.0,1.0,0.0,NaN,NaN,0,1,NaN,NaN,NaN
267,1125,1955,A138,2,12,0,1,0.0,1.0,0.0,...,0.0,0.0,0.0,NaN,NaN,0,0,NaN,NaN,NaN
286,1125,1955,A148,1,12,0,0,0.0,0.0,0.0,...,0.0,1.0,1.0,NaN,NaN,0,0,NaN,NaN,NaN
287,1125,1955,A148,2,8,0,0,0.0,0.0,0.0,...,0.0,0.0,2.0,NaN,NaN,0,0,NaN,NaN,NaN
288,1125,1955,A149,1,244,1,0,1.0,0.0,0.0,...,2.0,2.0,0.0,NaN,NaN,0,1,NaN,NaN,NaN
289,1125,1955,A149,2,31,1,0,0.0,0.0,0.0,...,0.0,0.0,0.0,NaN,NaN,0,1,NaN,NaN,NaN
558,1125,1956,A138,1,11,0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,NaN,NaN,0,0,NaN,NaN,NaN
559,1125,1956,A138,2,4,0,1,0.0,0.0,1.0,...,0.0,0.0,0.0,NaN,NaN,0,0,NaN,NaN,NaN
578,1125,1956,A148,1,12,0,0,0.0,0.0,0.0,...,0.0,1.0,0.0,NaN,NaN,0,0,NaN,NaN,NaN
579,1125,1956,A148,2,5,0,0,0.0,0.0,0.0,...,1.0,0.0,2.0,NaN,NaN,0,0,NaN,NaN,NaN


In [23]:
df1.duplicated(subset=["Country", "Year", "Cause", "Sex"]).any()   # should be false

False

In [24]:
df1 = extract(df1, concept_mapping, d7)

In [25]:
df1

deaths_all_ages  deaths_age_0_year  \
country year sex cause                                          
1125    1955 1   homicide              244                  1   
             2   homicide               31                  1   
        1956 1   homicide              288                  0   
             2   homicide               34                  0   
        1957 1   homicide              268                  3   
             2   homicide               34                  1   
        1958 1   homicide              215                  0   
             2   homicide               32                  0   
        1959 1   homicide              283                  2   
             2   homicide               48                  1   
...                                    ...                ...   
5150    1963 1   traffic               319                  2   
             2   traffic               101                  0   
        1964 1   traffic               328                  0   
             2   traffic               117                  2   
        1965 1   traffic               411                  2   
             2   traffic               140                  2   
        1966 1   traffic               427                  0   
             2   traffic               140                  1   
        1967 1   traffic               453                  4   
             2   traffic               167                  5   

                           deaths_age_1_year  deaths_age_2_years  \
country year sex cause                                             
1125    1955 1   homicide                  0                 1.0   
             2   homicide                  0                 0.0   
        1956 1   homicide                  0                 0.0   
             2   homicide                  0                 0.0   
        1957 1   homicide                  1                 0.0   
             2   homicide                  0                 1.0   
        1958 1   homicide                  1                 0.0   
             2   homicide                  0                 2.0   
        1959 1   homicide                  2                 0.0   
             2   homicide                  0                 1.0   
...                                      ...                 ...   
5150    1963 1   traffic                   2                 7.0   
             2   traffic                   4                 4.0   
        1964 1   traffic                   2                 3.0   
             2   traffic                   3                 1.0   
        1965 1   traffic                   2                 4.0   
             2   traffic                   2                 2.0   
        1966 1   traffic                   3                 4.0   
             2   traffic                   3                 3.0   
        1967 1   traffic                   4                 0.0   
             2   traffic                   4                 3.0   

                           deaths_age_3_years  deaths_age_4_years  \
country year sex cause                                              
1125    1955 1   homicide                 0.0                 0.0   
             2   homicide                 0.0                 0.0   
        1956 1   homicide                 2.0                 1.0   
             2   homicide                 1.0                 1.0   
        1957 1   homicide                 0.0                 1.0   
             2   homicide                 0.0                 0.0   
        1958 1   homicide                 0.0                 0.0   
             2   homicide                 0.0                 0.0   
        1959 1   homicide                 1.0                 0.0   
             2   homicide                 2.0                 0.0   
...                                       ...                 ...   
5150    1963 1   traffic                  3.0                 3.0   


In [26]:
icd8 = pd.read_csv("../source/morticd08.zip")

/Users/semio/.pyenv/versions/3.6.5/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [27]:
df2 = preprocess(icd8, d8.keys())

In [28]:
df2.duplicated(subset=["Country", "Year", "Cause", "Sex"]).any()   # should be false

False

In [29]:
df2 = extract(df2, concept_mapping, d8)

In [30]:
df2

deaths_all_ages  deaths_age_0_year  \
country year sex cause                                          
1060    1980 1   homicide                2                0.0   
             2   homicide                1                1.0   
1125    1973 1   homicide              408                8.0   
             2   homicide              130                5.0   
        1974 1   homicide              282                8.0   
             2   homicide              115                7.0   
        1975 1   homicide              320                9.0   
             2   homicide              107                8.0   
        1976 1   homicide              365                7.0   
             2   homicide              115                6.0   
...                                    ...                ...   
5150    1975 1   traffic               499                2.0   
             2   traffic               190                0.0   
        1976 1   traffic               493                6.0   
             2   traffic               170                0.0   
        1977 1   traffic               630                2.0   
             2   traffic               259                0.0   
        1978 1   traffic               481                2.0   
             2   traffic               207                2.0   
5195    1977 1   traffic                 9                1.0   
             2   traffic                 5                0.0   

                           deaths_age_1_year  deaths_age_2_years  \
country year sex cause                                             
1060    1980 1   homicide                0.0                 0.0   
             2   homicide                0.0                 0.0   
1125    1973 1   homicide                4.0                 4.0   
             2   homicide                4.0                 5.0   
        1974 1   homicide                0.0                 5.0   
             2   homicide                0.0                 3.0   
        1975 1   homicide                5.0                 2.0   
             2   homicide                2.0                 3.0   
        1976 1   homicide                1.0                 1.0   
             2   homicide                2.0                 3.0   
...                                      ...                 ...   
5150    1975 1   traffic                 1.0                 2.0   
             2   traffic                 4.0                 4.0   
        1976 1   traffic                 2.0                 3.0   
             2   traffic                 1.0                 4.0   
        1977 1   traffic                 2.0                 1.0   
             2   traffic                 2.0                 3.0   
        1978 1   traffic                 5.0                 3.0   
             2   traffic                 3.0                 3.0   
5195    1977 1   traffic                 0.0                 0.0   
             2   traffic                 0.0                 0.0   

                           deaths_age_3_years  deaths_age_4_years  \
country year sex cause                                              
1060    1980 1   homicide                 0.0                 0.0   
             2   homicide                 0.0                 0.0   
1125    1973 1   homicide                 7.0                 4.0   
             2   homicide                 5.0                 2.0   
        1974 1   homicide                 2.0                 1.0   
             2   homicide                 1.0                 2.0   
        1975 1   homicide                 5.0                 3.0   
             2   homicide                 3.0                 2.0   
        1976 1   homicide                 2.0                 5.0   
             2   homicide                 1.0                 1.0   
...                                       ...                 ...   
5150    1975 1   traffic                  7.0                 7.0   


In [31]:
icd9 = pd.read_csv('../source/morticd9.zip')

/Users/semio/.pyenv/versions/3.6.5/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [32]:
df3 = preprocess(icd9, d9.keys())

In [33]:
df3.duplicated(subset=["Country", "Year", "Cause", "Sex"]).any()   # should be False

False

In [34]:
df3.Cause.unique()

array(['B471', 'B54', 'B55', 'B472', 'B529'], dtype=object)

In [35]:
df3 = extract(df3, concept_mapping, d9)

In [36]:
df3

deaths_all_ages  deaths_age_0_year  \
country year sex cause                                          
1125    1980 1   homicide              294                3.0   
             2   homicide               77                2.0   
        1987 1   homicide              191                0.0   
             2   homicide               50                0.0   
1300    1981 1   homicide                6                0.0   
             2   homicide                1                0.0   
        1982 1   homicide               10                1.0   
             2   homicide                3                0.0   
        1983 1   homicide               11                0.0   
             2   homicide                1                0.0   
...                                    ...                ...   
5150    1996 1   traffic               418               11.0   
             2   traffic               180                6.0   
        1997 1   traffic               446               16.0   
             2   traffic               191                5.0   
        1998 1   traffic               409                9.0   
             2   traffic               195                7.0   
        1999 1   traffic               399                4.0   
             2   traffic               218                7.0   
5195    1980 1   traffic                29                0.0   
             2   traffic                 9                0.0   

                           deaths_age_1_year  deaths_age_2_years  \
country year sex cause                                             
1125    1980 1   homicide                1.0                 1.0   
             2   homicide                0.0                 0.0   
        1987 1   homicide                1.0                 0.0   
             2   homicide                0.0                 0.0   
1300    1981 1   homicide                0.0                 0.0   
             2   homicide                0.0                 0.0   
        1982 1   homicide                0.0                 0.0   
             2   homicide                0.0                 0.0   
        1983 1   homicide                0.0                 0.0   
             2   homicide                0.0                 0.0   
...                                      ...                 ...   
5150    1996 1   traffic                 3.0                 1.0   
             2   traffic                 2.0                 2.0   
        1997 1   traffic                 1.0                 5.0   
             2   traffic                 4.0                 4.0   
        1998 1   traffic                 4.0                 1.0   
             2   traffic                 1.0                 0.0   
        1999 1   traffic                 4.0                 2.0   
             2   traffic                 0.0                 1.0   
5195    1980 1   traffic                 0.0                 0.0   
             2   traffic                 0.0                 0.0   

                           deaths_age_3_years  deaths_age_4_years  \
country year sex cause                                              
1125    1980 1   homicide                 0.0                 4.0   
             2   homicide                 0.0                 0.0   
        1987 1   homicide                 0.0                 2.0   
             2   homicide                 0.0                 2.0   
1300    1981 1   homicide                 0.0                 0.0   
             2   homicide                 0.0                 0.0   
        1982 1   homicide                 0.0                 0.0   
             2   homicide                 0.0                 0.0   
        1983 1   homicide                 0.0                 0.0   
             2   homicide                 0.0                 0.0   
...                                       ...                 ...   
5150    1996 1   traffic                  1.0                 3.0   


In [37]:
icd101 = pd.read_csv('../source/Morticd10_part1.zip')

/Users/semio/.pyenv/versions/3.6.5/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [38]:
df4 = preprocess(icd101, d10.keys())

In [39]:
df4.duplicated(subset=["Country", "Year", "Cause", "Sex"]).any()   # should be False

False

In [40]:
df4 = extract(df4, concept_mapping, d10)

In [41]:
df4

deaths_all_ages  deaths_age_0_year  \
country year sex cause                                          
1125    2000 1   homicide               39                0.0   
             2   homicide               11                0.0   
        2001 1   homicide               55                0.0   
             2   homicide                8                0.0   
        2002 1   homicide               77                0.0   
             2   homicide                9                0.0   
        2003 1   homicide              124                0.0   
             2   homicide               11                0.0   
        2004 1   homicide               54                1.0   
             2   homicide               20                0.0   
...                                    ...                ...   
5150    2000 1   traffic               302                1.0   
             2   traffic               148                1.0   
        2001 1   traffic               304                1.0   
             2   traffic               150                0.0   
        2002 1   traffic               297                1.0   
             2   traffic               142                1.0   
        2003 1   traffic               325                0.0   
             2   traffic               155                1.0   
        2004 1   traffic               305                3.0   
             2   traffic               142                2.0   

                           deaths_age_1_year  deaths_age_2_years  \
country year sex cause                                             
1125    2000 1   homicide                2.0                 0.0   
             2   homicide                0.0                 0.0   
        2001 1   homicide                3.0                 0.0   
             2   homicide                1.0                 0.0   
        2002 1   homicide                0.0                 0.0   
             2   homicide                1.0                 0.0   
        2003 1   homicide                4.0                 0.0   
             2   homicide                0.0                 0.0   
        2004 1   homicide                1.0                 0.0   
             2   homicide                5.0                 0.0   
...                                      ...                 ...   
5150    2000 1   traffic                 1.0                 3.0   
             2   traffic                 1.0                 0.0   
        2001 1   traffic                 1.0                 1.0   
             2   traffic                 1.0                 0.0   
        2002 1   traffic                 0.0                 1.0   
             2   traffic                 1.0                 1.0   
        2003 1   traffic                 0.0                 1.0   
             2   traffic                 1.0                 1.0   
        2004 1   traffic                 1.0                 0.0   
             2   traffic                 1.0                 0.0   

                           deaths_age_3_years  deaths_age_4_years  \
country year sex cause                                              
1125    2000 1   homicide                 0.0                 0.0   
             2   homicide                 0.0                 0.0   
        2001 1   homicide                 0.0                 0.0   
             2   homicide                 0.0                 0.0   
        2002 1   homicide                 0.0                 0.0   
             2   homicide                 0.0                 0.0   
        2003 1   homicide                 0.0                 0.0   
             2   homicide                 0.0                 0.0   
        2004 1   homicide                 0.0                 0.0   
             2   homicide                 0.0                 0.0   
...                                       ...                 ...   
5150    2000 1   traffic                  4.0                 0.0   


In [42]:
icd102 = pd.read_csv('../source/Morticd10_part2.zip')

/Users/semio/.pyenv/versions/3.6.5/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [43]:
df5 = preprocess(icd102, d10.keys())

In [44]:
df5.duplicated(subset=["Country", "Year", "Cause", "Sex"]).any()   # should be False

False

In [45]:
df5 = extract(df5, concept_mapping, d10)

In [46]:
df5

deaths_all_ages  deaths_age_0_year  \
country year sex cause                                          
1060    2011 1   homicide               44                0.0   
             2   homicide                7                0.0   
        2012 1   homicide               45                1.0   
             2   homicide               10                0.0   
1125    2005 1   homicide               65                2.0   
             2   homicide               19                0.0   
        2006 1   homicide               59                0.0   
             2   homicide               16                1.0   
        2007 1   homicide               51                0.0   
             2   homicide                9                1.0   
...                                    ...                ...   
5150    2009 1   traffic               280                0.0   
             2   traffic               131                0.0   
        2010 1   traffic               288                0.0   
             2   traffic               114                1.0   
        2011 1   traffic               229                0.0   
             2   traffic                90                0.0   
        2012 1   traffic               241                0.0   
             2   traffic                93                0.0   
        2013 1   traffic               192                2.0   
             2   traffic                81                0.0   

                           deaths_age_1_year  deaths_age_2_years  \
country year sex cause                                             
1060    2011 1   homicide                1.0                 0.0   
             2   homicide                0.0                 0.0   
        2012 1   homicide                0.0                 0.0   
             2   homicide                0.0                 0.0   
1125    2005 1   homicide                4.0                 0.0   
             2   homicide                1.0                 0.0   
        2006 1   homicide                3.0                 0.0   
             2   homicide                1.0                 0.0   
        2007 1   homicide                1.0                 0.0   
             2   homicide                0.0                 0.0   
...                                      ...                 ...   
5150    2009 1   traffic                 2.0                 0.0   
             2   traffic                 0.0                 0.0   
        2010 1   traffic                 1.0                 1.0   
             2   traffic                 0.0                 1.0   
        2011 1   traffic                 0.0                 1.0   
             2   traffic                 0.0                 0.0   
        2012 1   traffic                 1.0                 0.0   
             2   traffic                 0.0                 1.0   
        2013 1   traffic                 0.0                 2.0   
             2   traffic                 0.0                 0.0   

                           deaths_age_3_years  deaths_age_4_years  \
country year sex cause                                              
1060    2011 1   homicide                 0.0                 0.0   
             2   homicide                 0.0                 0.0   
        2012 1   homicide                 0.0                 0.0   
             2   homicide                 0.0                 0.0   
1125    2005 1   homicide                 0.0                 0.0   
             2   homicide                 0.0                 0.0   
        2006 1   homicide                 0.0                 0.0   
             2   homicide                 0.0                 0.0   
        2007 1   homicide                 0.0                 0.0   
             2   homicide                 0.0                 0.0   
...                                       ...                 ...   
5150    2009 1   traffic                  4.0                 2.0   


In [47]:
df_all = pd.concat([df1, df2, df3, df4, df5])

In [48]:
df_all

deaths_all_ages  deaths_age_0_year  \
country year sex cause                                          
1125    1955 1   homicide              244                1.0   
             2   homicide               31                1.0   
        1956 1   homicide              288                0.0   
             2   homicide               34                0.0   
        1957 1   homicide              268                3.0   
             2   homicide               34                1.0   
        1958 1   homicide              215                0.0   
             2   homicide               32                0.0   
        1959 1   homicide              283                2.0   
             2   homicide               48                1.0   
...                                    ...                ...   
5150    2009 1   traffic               280                0.0   
             2   traffic               131                0.0   
        2010 1   traffic               288                0.0   
             2   traffic               114                1.0   
        2011 1   traffic               229                0.0   
             2   traffic                90                0.0   
        2012 1   traffic               241                0.0   
             2   traffic                93                0.0   
        2013 1   traffic               192                2.0   
             2   traffic                81                0.0   

                           deaths_age_1_year  deaths_age_2_years  \
country year sex cause                                             
1125    1955 1   homicide                0.0                 1.0   
             2   homicide                0.0                 0.0   
        1956 1   homicide                0.0                 0.0   
             2   homicide                0.0                 0.0   
        1957 1   homicide                1.0                 0.0   
             2   homicide                0.0                 1.0   
        1958 1   homicide                1.0                 0.0   
             2   homicide                0.0                 2.0   
        1959 1   homicide                2.0                 0.0   
             2   homicide                0.0                 1.0   
...                                      ...                 ...   
5150    2009 1   traffic                 2.0                 0.0   
             2   traffic                 0.0                 0.0   
        2010 1   traffic                 1.0                 1.0   
             2   traffic                 0.0                 1.0   
        2011 1   traffic                 0.0                 1.0   
             2   traffic                 0.0                 0.0   
        2012 1   traffic                 1.0                 0.0   
             2   traffic                 0.0                 1.0   
        2013 1   traffic                 0.0                 2.0   
             2   traffic                 0.0                 0.0   

                           deaths_age_3_years  deaths_age_4_years  \
country year sex cause                                              
1125    1955 1   homicide                 0.0                 0.0   
             2   homicide                 0.0                 0.0   
        1956 1   homicide                 2.0                 1.0   
             2   homicide                 1.0                 1.0   
        1957 1   homicide                 0.0                 1.0   
             2   homicide                 0.0                 0.0   
        1958 1   homicide                 0.0                 0.0   
             2   homicide                 0.0                 0.0   
        1959 1   homicide                 1.0                 0.0   
             2   homicide                 2.0                 0.0   
...                                       ...                 ...   
5150    2009 1   traffic                  4.0                 2.0   


In [49]:
# population

from http://apps.who.int/healthinfo/statistics/mortality/whodpms/definitions/pop.htm

In [50]:
pop_div = pd.read_csv("../source/pop.csv")

In [51]:
pop_div

,All ages,"100,000"
0,0,1822.0
1,01-04,7033.0
2,05-09,8687.0
3,10-14,8597.0
4,15-19,8474.0
5,20-24,8222.0
6,25-29,7928.0
7,30-34,7605.0
8,35-39,7145.0
9,40-44,6590.0


In [52]:
pop_concepts = pd.read_csv('../source/concepts_pop.csv')

In [53]:
pop_concepts = pop_concepts.dropna(how='all', axis=1).copy()

In [54]:
pop_concepts

,Column name,Content
0,Country,Country”
1,Year,Year to which data refer
2,Sex,"1 male, 2 female"
3,Pop1,Population at all ages
4,Pop2,Population at age 0 year
5,Pop3,Population at age1 year
6,Pop4,Population at age 2 years
7,Pop5,Population at age 3 years
8,Pop6,Population at age 4 years
9,Pop7,Population at age 5-9 years


In [55]:
from ddf_utils.str import to_concept_id

In [56]:
pop_concepts['concept'] = pop_concepts['Column name'].map(to_concept_id)

In [57]:
pop_concepts.loc[3:, 'concept'] = pop_concepts.loc[3:, 'Content'].map(lambda x: to_concept_id(x.replace("at ", "")))

In [58]:
pop_concepts

,Column name,Content,concept
0,Country,Country”,country
1,Year,Year to which data refer,year
2,Sex,"1 male, 2 female",sex
3,Pop1,Population at all ages,population_all_ages
4,Pop2,Population at age 0 year,population_age_0_year
5,Pop3,Population at age1 year,population_age1_year
6,Pop4,Population at age 2 years,population_age_2_years
7,Pop5,Population at age 3 years,population_age_3_years
8,Pop6,Population at age 4 years,population_age_4_years
9,Pop7,Population at age 5-9 years,population_age_5_9_years


In [59]:
pop_concept_mapping = pop_concepts.set_index('Column name')['concept'].to_dict()

In [60]:
pop_concept_mapping

{'Country': 'country',
 'Year': 'year',
 'Sex': 'sex',
 'Pop1': 'population_all_ages',
 'Pop2': 'population_age_0_year',
 'Pop3': 'population_age1_year',
 'Pop4': 'population_age_2_years',
 'Pop5': 'population_age_3_years',
 'Pop6': 'population_age_4_years',
 'Pop7': 'population_age_5_9_years',
 'Pop8': 'population_age_10_14_years',
 'Pop9': 'population_age_15_19_years',
 'Pop10': 'population_age_20_24_years',
 'Pop11': 'population_age_25_29_years',
 'Pop12': 'population_age_30_34_years',
 'Pop13': 'population_age_35_39_years',
 'Pop14': 'population_age_40_44_years',
 'Pop15': 'population_age_45_49_years',
 'Pop16': 'population_age_50_54_years',
 'Pop17': 'population_age_55_59_years',
 'Pop18': 'population_age_60_64_years',
 'Pop19': 'population_age_65_69_years',
 'Pop20': 'population_age_70_74_years',
 'Pop21': 'population_age_75_79_years',
 'Pop22': 'population_age_80_84_years',
 'Pop23': 'population_age_85_89_years',
 'Pop24': 'population_age_90_94_years',
 'Pop25': 'population_age_

In [61]:
pop = pd.read_csv('../source/Pop.zip')

In [62]:
from etl import is_country

In [63]:
pop

,Country,Admin1,SubDiv,Year,Sex,Frmat,Pop1,Pop2,Pop3,Pop4,...,Pop18,Pop19,Pop20,Pop21,Pop22,Pop23,Pop24,Pop25,Pop26,Lb
0,1060,NaN,NaN,1980,1,7,137100.0,3400.0,15800.0,NaN,...,NaN,5300.0,NaN,2900.0,NaN,NaN,NaN,NaN,6500.0,5000.0
1,1060,NaN,NaN,1980,2,7,159000.0,4000.0,18400.0,NaN,...,NaN,6200.0,NaN,3400.0,NaN,NaN,NaN,NaN,7500.0,6000.0
2,1125,NaN,NaN,1955,1,2,5051500.0,150300.0,543400.0,NaN,...,110200.0,51100.0,41600.0,14300.0,11800.0,25300.0,NaN,NaN,0.0,253329.0
3,1125,NaN,NaN,1955,2,2,5049400.0,145200.0,551000.0,NaN,...,122100.0,51100.0,50700.0,15800.0,18000.0,28500.0,NaN,NaN,0.0,237901.0
4,1125,NaN,NaN,1956,1,2,5353700.0,158700.0,576600.0,NaN,...,116900.0,54100.0,44000.0,14900.0,12400.0,26600.0,NaN,NaN,0.0,250022.0
5,1125,NaN,NaN,1956,2,2,5351400.0,153600.0,584800.0,NaN,...,129500.0,54300.0,53800.0,16700.0,19000.0,29400.0,NaN,NaN,0.0,235887.0
6,1125,NaN,NaN,1957,1,2,5403000.0,160300.0,580800.0,NaN,...,118400.0,54100.0,44100.0,14900.0,12400.0,26500.0,NaN,NaN,0.0,231412.0
7,1125,NaN,NaN,1957,2,2,5392000.0,155300.0,589400.0,NaN,...,130000.0,54000.0,53500.0,16400.0,18700.0,29800.0,NaN,NaN,0.0,215811.0
8,1125,NaN,NaN,1958,1,2,5506900.0,162800.0,592000.0,NaN,...,120600.0,54600.0,44500.0,15100.0,12500.0,26700.0,NaN,NaN,0.0,255600.0
9,1125,NaN,NaN,1958,2,2,5494400.0,157800.0,600100.0,NaN,...,132400.0,54700.0,54100.0,16600.0,18800.0,30200.0,NaN,NaN,0.0,238109.0


In [64]:
pop.columns

Index(['Country', 'Admin1', 'SubDiv', 'Year', 'Sex', 'Frmat', 'Pop1', 'Pop2',
       'Pop3', 'Pop4', 'Pop5', 'Pop6', 'Pop7', 'Pop8', 'Pop9', 'Pop10',
       'Pop11', 'Pop12', 'Pop13', 'Pop14', 'Pop15', 'Pop16', 'Pop17', 'Pop18',
       'Pop19', 'Pop20', 'Pop21', 'Pop22', 'Pop23', 'Pop24', 'Pop25', 'Pop26',
       'Lb'],
      dtype='object')

In [65]:
pop["is_country"] = is_country(pop)

In [66]:
pop = pop[pop.is_country == True]

In [67]:
cols = ['Country', 'Year', 'Sex', 'Pop1', 'Pop2',
       'Pop3', 'Pop4', 'Pop5', 'Pop6', 'Pop7', 'Pop8', 'Pop9', 'Pop10',
       'Pop11', 'Pop12', 'Pop13', 'Pop14', 'Pop15', 'Pop16', 'Pop17', 'Pop18',
       'Pop19', 'Pop20', 'Pop21', 'Pop22', 'Pop23', 'Pop24', 'Pop25', 'Pop26',
       'Lb']

In [68]:
pop = pop[cols].copy()

In [69]:
pop.columns = pop.columns.map(pop_concept_mapping)

In [70]:
pop.columns

Index(['country', 'year', 'sex', 'population_all_ages',
       'population_age_0_year', 'population_age1_year',
       'population_age_2_years', 'population_age_3_years',
       'population_age_4_years', 'population_age_5_9_years',
       'population_age_10_14_years', 'population_age_15_19_years',
       'population_age_20_24_years', 'population_age_25_29_years',
       'population_age_30_34_years', 'population_age_35_39_years',
       'population_age_40_44_years', 'population_age_45_49_years',
       'population_age_50_54_years', 'population_age_55_59_years',
       'population_age_60_64_years', 'population_age_65_69_years',
       'population_age_70_74_years', 'population_age_75_79_years',
       'population_age_80_84_years', 'population_age_85_89_years',
       'population_age_90_94_years', 'population_age_95_years_and_over',
       'population_age_unspecified', 'live_births'],
      dtype='object')

In [71]:
pop

,country,year,sex,population_all_ages,population_age_0_year,population_age1_year,population_age_2_years,population_age_3_years,population_age_4_years,population_age_5_9_years,...,population_age_60_64_years,population_age_65_69_years,population_age_70_74_years,population_age_75_79_years,population_age_80_84_years,population_age_85_89_years,population_age_90_94_years,population_age_95_years_and_over,population_age_unspecified,live_births
0,1060,1980,1,137100.0,3400.0,15800.0,NaN,NaN,NaN,39200.0,...,NaN,5300.0,NaN,2900.0,NaN,NaN,NaN,NaN,6500.0,5000.0
1,1060,1980,2,159000.0,4000.0,18400.0,NaN,NaN,NaN,45400.0,...,NaN,6200.0,NaN,3400.0,NaN,NaN,NaN,NaN,7500.0,6000.0
2,1125,1955,1,5051500.0,150300.0,543400.0,NaN,NaN,NaN,615300.0,...,110200.0,51100.0,41600.0,14300.0,11800.0,25300.0,NaN,NaN,0.0,253329.0
3,1125,1955,2,5049400.0,145200.0,551000.0,NaN,NaN,NaN,613400.0,...,122100.0,51100.0,50700.0,15800.0,18000.0,28500.0,NaN,NaN,0.0,237901.0
4,1125,1956,1,5353700.0,158700.0,576600.0,NaN,NaN,NaN,653000.0,...,116900.0,54100.0,44000.0,14900.0,12400.0,26600.0,NaN,NaN,0.0,250022.0
5,1125,1956,2,5351400.0,153600.0,584800.0,NaN,NaN,NaN,650900.0,...,129500.0,54300.0,53800.0,16700.0,19000.0,29400.0,NaN,NaN,0.0,235887.0
6,1125,1957,1,5403000.0,160300.0,580800.0,NaN,NaN,NaN,657600.0,...,118400.0,54100.0,44100.0,14900.0,12400.0,26500.0,NaN,NaN,0.0,231412.0
7,1125,1957,2,5392000.0,155300.0,589400.0,NaN,NaN,NaN,655400.0,...,130000.0,54000.0,53500.0,16400.0,18700.0,29800.0,NaN,NaN,0.0,215811.0
8,1125,1958,1,5506900.0,162800.0,592000.0,NaN,NaN,NaN,670000.0,...,120600.0,54600.0,44500.0,15100.0,12500.0,26700.0,NaN,NaN,0.0,255600.0
9,1125,1958,2,5494400.0,157800.0,600100.0,NaN,NaN,NaN,667400.0,...,132400.0,54700.0,54100.0,16600.0,18800.0,30200.0,NaN,NaN,0.0,238109.0


In [72]:
pop_ = pop.copy()

In [73]:
pop_div

,All ages,"100,000"
0,0,1822.0
1,01-04,7033.0
2,05-09,8687.0
3,10-14,8597.0
4,15-19,8474.0
5,20-24,8222.0
6,25-29,7928.0
7,30-34,7605.0
8,35-39,7145.0
9,40-44,6590.0


In [74]:
pop_div_t = pop_div.set_index('All ages').T.reset_index(drop=True)

In [75]:
pop_div_t

All ages,0,01-04,05-09,10-14,15-19,20-24,25-29,30-34,35-39,40-44,45-49,50-54,55-59,60-64,65-69,70-74,75-79,80-84,85+
0,1822.0,7033.0,8687.0,8597.0,8474.0,8222.0,7928.0,7605.0,7145.0,6590.0,6038.0,5371.0,4547.0,3723.0,2955.0,2210.0,1515.0,905.0,632.0


In [76]:
pop_div_t.sum(axis=1)

0    99999.0
dtype: float64

In [77]:
pop_ = pop_.rename(columns={'population_age1_year': 'population_age_1_year'})
pop_ = pop_.set_index(['country', 'year', 'sex'])

In [78]:
# serving

In [79]:
from etl import rate

In [80]:
pop_.columns

Index(['population_all_ages', 'population_age_0_year', 'population_age_1_year',
       'population_age_2_years', 'population_age_3_years',
       'population_age_4_years', 'population_age_5_9_years',
       'population_age_10_14_years', 'population_age_15_19_years',
       'population_age_20_24_years', 'population_age_25_29_years',
       'population_age_30_34_years', 'population_age_35_39_years',
       'population_age_40_44_years', 'population_age_45_49_years',
       'population_age_50_54_years', 'population_age_55_59_years',
       'population_age_60_64_years', 'population_age_65_69_years',
       'population_age_70_74_years', 'population_age_75_79_years',
       'population_age_80_84_years', 'population_age_85_89_years',
       'population_age_90_94_years', 'population_age_95_years_and_over',
       'population_age_unspecified', 'live_births'],
      dtype='object')

In [81]:
gs014 = [
        {'spec': ['population_age_0_year'], 'std': '0'},
        {
            'spec': [
                'population_age_1_year',
                'population_age_2_years', 'population_age_3_years',
                'population_age_4_years'
            ],
            'std': '01-04'},
        {'spec': '5_9', 'std': '05-09'},
        {'spec': '10_14', 'std': '10-14'}
    ]

gs1529 = [
        {'spec': '15_19', 'std': '15-19'},
        {'spec': '20_24', 'std': '20-24'},
        {'spec': '25_29', 'std': '25-29'}
    ]

gs3044 = [
        {'spec': '30_34', 'std': '30-34'},
        {'spec': '35_39', 'std': '35-39'},
        {'spec': '40_44', 'std': '40-44'}
    ]

gs4559 = [
        {'spec': '45_49', 'std': '45-49'},
        {'spec': '50_54', 'std': '50-54'},
        {'spec': '55_59', 'std': '55-59'}
    ]

gs60plus = [
        {'spec': '60_64', 'std': '60-64'},
        {'spec': '65_69', 'std': '65-69'},
        {'spec': '70_74', 'std': '70-74'},
        {'spec': '75_79', 'std': '75-79'},
        {'spec': '80_84', 'std': '80-84'},
        {'spec': [
            'population_age_85_89_years',
            'population_age_90_94_years',
            'population_age_95_years_and_over'
        ], 'std': '85+'}
    ]

In [82]:
# creat both_sex for sex entity

In [100]:
deaths = df_all.copy()

In [101]:
deaths_bothsex = deaths.groupby(level=[0, 1, 3]).sum(min_count=2)  # only consider data avaliable in both sexes

In [102]:
deaths_bothsex['sex'] = 0
deaths_bothsex = deaths_bothsex.set_index('sex', append=True)

In [103]:
deaths_bothsex

deaths_all_ages  deaths_age_0_year  \
country year cause    sex                                       
1060    1980 homicide 0                3.0                1.0   
             suicide  0                7.0                0.0   
             traffic  0                8.0                0.0   
        2011 homicide 0               51.0                0.0   
             suicide  0               42.0                0.0   
        2012 homicide 0               55.0                1.0   
             suicide  0               45.0                0.0   
1125    1955 homicide 0              275.0                2.0   
             suicide  0               20.0                0.0   
             traffic  0               78.0                1.0   
...                                    ...                ...   
5150    2012 traffic  0              334.0                0.0   
        2013 homicide 0               52.0                0.0   
             suicide  0              513.0                0.0   
             traffic  0              273.0                2.0   
5195    1977 homicide 0                4.0                0.0   
             suicide  0                3.0                0.0   
             traffic  0               14.0                1.0   
        1980 homicide 0                4.0                0.0   
             suicide  0                5.0                0.0   
             traffic  0               38.0                0.0   

                           deaths_age_1_year  deaths_age_2_years  \
country year cause    sex                                          
1060    1980 homicide 0                  0.0                 0.0   
             suicide  0                  0.0                 0.0   
             traffic  0                  1.0                 0.0   
        2011 homicide 0                  1.0                 0.0   
             suicide  0                  0.0                 0.0   
        2012 homicide 0                  0.0                 0.0   
             suicide  0                  0.0                 0.0   
1125    1955 homicide 0                  0.0                 1.0   
             suicide  0                  0.0                 0.0   
             traffic  0                  1.0                 0.0   
...                                      ...                 ...   
5150    2012 traffic  0                  1.0                 1.0   
        2013 homicide 0                  0.0                 0.0   
             suicide  0                  0.0                 0.0   
             traffic  0                  0.0                 2.0   
5195    1977 homicide 0                  0.0                 1.0   
             suicide  0                  0.0                 0.0   
             traffic  0                  0.0                 0.0   
        1980 homicide 0                  0.0                 0.0   
             suicide  0                  0.0                 0.0   
             traffic  0                  0.0                 0.0   

                           deaths_age_3_years  deaths_age_4_years  \
country year cause    sex                                           
1060    1980 homicide 0                   0.0                 0.0   
             suicide  0                   0.0                 0.0   
             traffic  0                   0.0                 0.0   
        2011 homicide 0                   0.0                 0.0   
             suicide  0                   0.0                 0.0   
        2012 homicide 0                   0.0                 0.0   
             suicide  0                   0.0                 0.0   
1125    1955 homicide 0                   0.0                 0.0   
             suicide  0                   0.0                 0.0   
             traffic  0                   4.0                 2.0   
...                                       ...                 ...   
5150    2012 traffic  0                   0.0                 0.0   


In [104]:
deaths = deaths.append(deaths_bothsex)

In [105]:
deaths = deaths.sort_index()

In [106]:
deaths

deaths_all_ages  deaths_age_0_year  \
country year sex      cause                                          
1060    1980 1        homicide              2.0                0.0   
                      suicide               6.0                0.0   
                      traffic               3.0                0.0   
             2        homicide              1.0                1.0   
                      suicide               1.0                0.0   
                      traffic               5.0                0.0   
             homicide 0                     3.0                1.0   
             suicide  0                     7.0                0.0   
             traffic  0                     8.0                0.0   
        2011 1        homicide             44.0                0.0   
...                                         ...                ...   
5195    1977 traffic  0                    14.0                1.0   
        1980 1        homicide              3.0                0.0   
                      suicide               2.0                0.0   
                      traffic              29.0                0.0   
             2        homicide              1.0                0.0   
                      suicide               3.0                0.0   
                      traffic               9.0                0.0   
             homicide 0                     4.0                0.0   
             suicide  0                     5.0                0.0   
             traffic  0                    38.0                0.0   

                                deaths_age_1_year  deaths_age_2_years  \
country year sex      cause                                             
1060    1980 1        homicide                0.0                 0.0   
                      suicide                 0.0                 0.0   
                      traffic                 0.0                 0.0   
             2        homicide                0.0                 0.0   
                      suicide                 0.0                 0.0   
                      traffic                 1.0                 0.0   
             homicide 0                       0.0                 0.0   
             suicide  0                       0.0                 0.0   
             traffic  0                       1.0                 0.0   
        2011 1        homicide                1.0                 0.0   
...                                           ...                 ...   
5195    1977 traffic  0                       0.0                 0.0   
        1980 1        homicide                0.0                 0.0   
                      suicide                 0.0                 0.0   
                      traffic                 0.0                 0.0   
             2        homicide                0.0                 0.0   
                      suicide                 0.0                 0.0   
                      traffic                 0.0                 0.0   
             homicide 0                       0.0                 0.0   
             suicide  0                       0.0                 0.0   
             traffic  0                       0.0                 0.0   

                                deaths_age_3_years  deaths_age_4_years  \
country year sex      cause                                              
1060    1980 1        homicide                 0.0                 0.0   
                      suicide                  0.0                 0.0   
                      traffic                  0.0                 0.0   
             2        homicide                 0.0                 0.0   
                      suicide                  0.0                 0.0   
                      traffic                  0.0                 0.0   
             homicide 0                        0.0                 0.0   
             suicide  0                        0.0                 0.0   
 

In [113]:
deaths = deaths.reorder_levels(['country', 'year', 'cause', 'sex'])

In [107]:
pop_bothsex = pop_.groupby(level=[0, 1]).agg('sum', min_count=2)

In [108]:
pop_bothsex['sex'] = 0
pop_bothsex = pop_bothsex.set_index('sex', append=True)

In [109]:
pop_ = pop_.append(pop_bothsex)
pop_ = pop_.sort_index()

In [110]:
pop_

population_all_ages  population_age_0_year  \
country year sex                                               
1060    1980 0               296100.0                 7400.0   
             1               137100.0                 3400.0   
             2               159000.0                 4000.0   
1125    1955 0             10100900.0               295500.0   
             1              5051500.0               150300.0   
             2              5049400.0               145200.0   
        1956 0             10705100.0               312300.0   
             1              5353700.0               158700.0   
             2              5351400.0               153600.0   
        1957 0             10795000.0               315600.0   
...                               ...                    ...   
5150    2013 2              2269800.0                29380.0   
5195    1977 0              2809000.0               108800.0   
             1              1451000.0                55400.0   
             2              1358000.0                53400.0   
        1980 0              3010700.0               102700.0   
             1              1575700.0                53400.0   
             2              1435000.0                49300.0   
5200    2003 0               101404.0                 2221.0   
             1                51711.0                 1156.0   
             2                49693.0                 1065.0   

                  population_age_1_year  population_age_2_years  \
country year sex                                                  
1060    1980 0                  34200.0                     NaN   
             1                  15800.0                     NaN   
             2                  18400.0                     NaN   
1125    1955 0                1094400.0                     NaN   
             1                 543400.0                     NaN   
             2                 551000.0                     NaN   
        1956 0                1161400.0                     NaN   
             1                 576600.0                     NaN   
             2                 584800.0                     NaN   
        1957 0                1170200.0                     NaN   
...                                 ...                     ...   
5150    2013 2                  29960.0                 30590.0   
5195    1977 0                 392100.0                     NaN   
             1                 199900.0                     NaN   
             2                 192200.0                     NaN   
        1980 0                  90600.0                 93600.0   
             1                  47400.0                 48900.0   
             2                  43200.0                 44700.0   
5200    2003 0                   9758.0                     NaN   
             1                   5045.0                     NaN   
             2                   4713.0                     NaN   

                  population_age_3_years  population_age_4_years  \
country year sex                                                   
1060    1980 0                       NaN                     NaN   
             1                       NaN                     NaN   
             2                       NaN                     NaN   
1125    1955 0                       NaN                     NaN   
             1                       NaN                     NaN   
             2                       NaN                     NaN   
        1956 0                       NaN                     NaN   
             1                       NaN                     NaN   
             2                       NaN                     NaN   
        1957 0                       NaN                     NaN   
...                                  ...                     ...   
5150    2013 2                   31330.0                 30560.0   
5195    1977 0                       NaN           

In [112]:
gsall = [*gs014, *gs1529, *gs3044, *gs4559, *gs60plus]

In [114]:
group = deaths.groupby(level=2)

res = {}

for g, _ in group:
    df = group.get_group(g)
    df.index = df.index.droplevel(2)
    r = rate(df, pop_, pop_div_t,  gsall)
    res[g] = r

In [115]:
to_concat = []

for g, v in res.items():
    v = v.reset_index()
    v['cause'] = g
    v = v.set_index(['country', 'year', 'cause', 'sex'])
    to_concat.append(v)

In [116]:
rate_all = pd.concat(to_concat)

In [117]:
rate_all.dropna().reset_index().query('country == "1125"').year.unique()

array([1955., 1956., 1957., 1958., 1959., 1960., 1961., 1962., 1963.,
       2004., 2005., 2006., 2007., 2008., 2014., 2015.])

In [118]:
rate_all.columns = ['deaths_rate_all_ages']

In [119]:
from ddf_utils.str import format_float_digits

In [165]:
rate_all = (rate_all.replace([np.inf, -np.inf], np.nan)
            .dropna()
            .applymap(format_float_digits)
            .sort_index()
            .reset_index())

for c in ['country', 'year', 'sex']:
    rate_all[c] = rate_all[c].astype(int)


rate_all.to_csv('../../ddf--datapoints--deaths_rate_all_ages--by--country--year--cause--sex.csv', index=False)

In [122]:
# alternative rate_all

In [123]:
d = deaths['deaths_all_ages']
p = pop_['population_all_ages']

In [124]:
def _rate(df):
    df_ = df.copy()
    df_.index = df_.index.droplevel(2)
    return  df_ / p * 100000

In [125]:
gs_ = d.groupby(level=2)

res = []

for g_, _ in gs_:
    df = gs_.get_group(g_)
    df_new = _rate(df)
    df_new = df_new.reset_index()
    df_new['cause'] = g_
    df_new = df_new.set_index(['country', 'year', 'cause', 'sex'])
    res.append(df_new)

In [126]:
rate_all_alt = pd.concat(res)

In [127]:
rate_all_alt.columns = ['deaths_rate_all_ages_alt']

In [166]:
rate_all_alt = (rate_all_alt.replace([np.inf, -np.inf], np.nan)
                .dropna()
                .applymap(format_float_digits)
                .sort_index()
                .reset_index())

for c in ['country', 'year', 'sex']:
    rate_all_alt[c] = rate_all_alt[c].astype(int)

rate_all_alt.to_csv('../../ddf--datapoints--deaths_rate_all_ages_alt--by--country--year--cause--sex.csv', index=False)

In [129]:
# 014

group = deaths.groupby(level=2)

res = {}

for g, _ in group:
    df = group.get_group(g)
    df.index = df.index.droplevel(2)
    r = rate(df, pop_, pop_div_t,  gs014)
    res[g] = r
    
to_concat = []

for g, v in res.items():
    v = v.reset_index()
    v['cause'] = g
    v = v.set_index(['country', 'year', 'cause', 'sex'])
    to_concat.append(v)
    
rate_014 = pd.concat(to_concat)

In [130]:
rate_014.dropna().reset_index().query('country == "1125"').year.unique()

array([1955., 1956., 1957., 1958., 1959., 1960., 1961., 1962., 1963.,
       1964., 1965., 1973., 1974., 1975., 1976., 1977., 1978., 1979.,
       1980., 1987., 2001., 2002., 2003., 2004., 2005., 2006., 2007.,
       2008., 2009., 2010., 2011., 2012., 2013., 2014., 2015., 1966.,
       1967., 1970., 1971., 1972.])

In [131]:
rate_014.columns = ['deaths_rate_ages_0_14_years']

In [132]:
rate_014 = rate_014.reset_index()

for c in ['country', 'year', 'sex']:
    rate_014[c] = rate_014[c].map(lambda x: str(int(float(x))))
    
rate_014 = rate_014.set_index(['country', 'year', 'cause', 'sex'])

In [167]:
rate_014 = (rate_014.replace([np.inf, -np.inf], np.nan)
                .dropna()
                .applymap(format_float_digits)
                .sort_index()
                .reset_index())

for c in ['country', 'year', 'sex']:
    rate_014[c] = rate_014[c].astype(int)

rate_014.to_csv('../../ddf--datapoints--deaths_rate_ages_0_14_years--by--country--year--cause--sex.csv', index=False)

In [134]:
# 1529

In [135]:
group = deaths.groupby(level=2)

res = {}

for g, _ in group:
    df = group.get_group(g)
    df.index = df.index.droplevel(2)
    r = rate(df, pop_, pop_div_t,  gs1529)
    res[g] = r
    
to_concat = []

for g, v in res.items():
    v = v.reset_index()
    v['cause'] = g
    v = v.set_index(['country', 'year', 'cause', 'sex'])
    to_concat.append(v)
    
rate_1529 = pd.concat(to_concat)

In [136]:
rate_1529.dropna().reset_index().query('country == "1125"').year.unique()

array([1955., 1956., 1957., 1958., 1959., 1960., 1961., 1962., 1963.,
       1964., 1965., 1973., 1974., 1975., 1976., 1977., 1978., 1979.,
       1980., 1987., 2001., 2002., 2003., 2004., 2005., 2006., 2007.,
       2008., 2009., 2010., 2011., 2012., 2013., 2014., 2015., 1966.,
       1967., 1970., 1971., 1972.])

In [137]:
rate_1529.columns = ['deaths_rate_ages_15_29_years']

In [168]:
rate_1529 = (rate_1529.replace([np.inf, -np.inf], np.nan)
             .dropna()
             .applymap(format_float_digits)
             .sort_index()
             .reset_index())

for c in ['country', 'year', 'sex']:
    rate_1529[c] = rate_1529[c].astype(int)

rate_1529.to_csv('../../ddf--datapoints--deaths_rate_ages_15_29_years--by--country--year--cause--sex.csv', index=False)

In [169]:
for n1, n2 in zip([gs3044, gs4559, gs60plus], ['30_44', '45_59', '60plus']):
    group = deaths.groupby(level=2)

    res = {}

    for g, _ in group:
        df = group.get_group(g)
        df.index = df.index.droplevel(2)
        r = rate(df, pop_, pop_div_t,  n1)
        res[g] = r

    to_concat = []

    for g, v in res.items():
        v = v.reset_index()
        v['cause'] = g
        v = v.set_index(['country', 'year', 'cause', 'sex'])
        to_concat.append(v)

    rate_g = pd.concat(to_concat)
    
    rate_g.columns = [f'deaths_rate_ages_{n2}_years']
    
    rate_g = (rate_g.replace([np.inf, -np.inf], np.nan)
              .dropna()
              .applymap(format_float_digits)
              .sort_index()
              .reset_index())
    
    for c in ['country', 'year', 'sex']:
        rate_g[c] = rate_1529[c].astype(int)
        
    rate_g.to_csv(f'../../ddf--datapoints--deaths_rate_ages_{n2}_years--by--country--year--cause--sex.csv', 
                  index=False)

In [171]:
(deaths[['deaths_all_ages']]
 .query('sex in [0, 1, 2]')
 .applymap(int)
 .to_csv('../../ddf--datapoints--deaths_all_ages--by--country--year--cause--sex.csv'))

In [141]:
measures = [f'deaths_rate_ages_{x}_years' for x in ['0_14', '15_29', '30_44', '45_59', '60plus']]

In [142]:
measures.append('deaths_rate_all_ages')
measures.append('deaths_rate_all_ages_alt')
measures.append('deaths_all_ages')

In [143]:
names = [f'Age-standardized deaths rate per 100000 people aged {x} years' 
         for x in ['0-14', '15-29', '30-44', '45-59', '60plus']]

In [144]:
names.append('Age-standardized deaths rate for people all ages')
names.append('crude deaths rate for people all ages')
names.append('total deaths number for all ages')

In [145]:
measures_df = pd.DataFrame({'concept': measures})

In [146]:
measures_df['concept_type'] = 'measure'
measures_df['name'] = names

In [147]:
measures_df

,concept,concept_type,name
0,deaths_rate_ages_0_14_years,measure,Age-standardized deaths rate per 100000 people...
1,deaths_rate_ages_15_29_years,measure,Age-standardized deaths rate per 100000 people...
2,deaths_rate_ages_30_44_years,measure,Age-standardized deaths rate per 100000 people...
3,deaths_rate_ages_45_59_years,measure,Age-standardized deaths rate per 100000 people...
4,deaths_rate_ages_60plus_years,measure,Age-standardized deaths rate per 100000 people...
5,deaths_rate_all_ages,measure,Age-standardized deaths rate for people all ages
6,deaths_rate_all_ages_alt,measure,crude deaths rate for people all ages
7,deaths_all_ages,measure,total deaths number for all ages


In [148]:
discrete = [ 'country', 'sex', 'cause', 'year', 'name']
dt = ['entity_domain', 'entity_domain', 'entity_domain', 'time', 'string']

In [149]:
discrete_df = pd.DataFrame({'concept': discrete, 'concept_type': dt, 'name': [x.title() for x in discrete]})

In [150]:
discrete_df

,concept,concept_type,name
0,country,entity_domain,Country
1,sex,entity_domain,Sex
2,cause,entity_domain,Cause
3,year,time,Year
4,name,string,Name


In [151]:
cdf = pd.concat([discrete_df, measures_df], ignore_index=True)

In [152]:
cdf

,concept,concept_type,name
0,country,entity_domain,Country
1,sex,entity_domain,Sex
2,cause,entity_domain,Cause
3,year,time,Year
4,name,string,Name
5,deaths_rate_ages_0_14_years,measure,Age-standardized deaths rate per 100000 people...
6,deaths_rate_ages_15_29_years,measure,Age-standardized deaths rate per 100000 people...
7,deaths_rate_ages_30_44_years,measure,Age-standardized deaths rate per 100000 people...
8,deaths_rate_ages_45_59_years,measure,Age-standardized deaths rate per 100000 people...
9,deaths_rate_ages_60plus_years,measure,Age-standardized deaths rate per 100000 people...


In [153]:
cdf.to_csv('../../ddf--concepts.csv', index=False)

In [154]:
country = pd.read_csv('../source/country_codes.zip')

In [155]:
country.to_csv('../../ddf--entities--country.csv', index=False)

In [156]:
sex = pd.DataFrame({'sex': [0, 1, 2], 'name': ['both_sexes', 'male', 'female']})

In [157]:
sex

,sex,name
0,0,both_sexes
1,1,male
2,2,female


In [158]:
sex = sex.to_csv('../../ddf--entities--sex.csv', index=False)

In [159]:
cause = pd.DataFrame({'cause': ['traffic', 'suicide', 'homicide'], 
                      'name': ['Road traffic accidents', 'Self-inflicted injuries', 'Homicide']})

In [160]:
cause

,cause,name
0,traffic,Road traffic accidents
1,suicide,Self-inflicted injuries
2,homicide,Homicide


In [161]:
cause.to_csv('../../ddf--entities--cause.csv', index=False)

In [162]:
deaths.query('country == "2020" and cause == "homicide" and year == 1997')

deaths_all_ages  deaths_age_0_year  \
country year cause    sex                                       
2020    1997 homicide 1             1399.0               30.0   
                      2              256.0               16.0   
                      9               43.0                1.0   

                           deaths_age_1_year  deaths_age_2_years  \
country year cause    sex                                          
2020    1997 homicide 1                  4.0                 2.0   
                      2                  1.0                 9.0   
                      9                  0.0                 0.0   

                           deaths_age_3_years  deaths_age_4_years  \
country year cause    sex                                           
2020    1997 homicide 1                   0.0                 1.0   
                      2                   2.0                 1.0   
                      9                   0.0                 0.0   

                           deaths_age_5_9_years  deaths_age_10_14_years  \
country year cause    sex                                                 
2020    1997 homicide 1                     7.0                    23.0   
                      2                     4.0                    13.0   
                      9                     0.0                     0.0   

                           deaths_age_15_19_years  deaths_age_20_24_years  \
country year cause    sex                                                   
2020    1997 homicide 1                     179.0                   221.0   
                      2                      21.0                    23.0   
                      9                       7.0                     4.0   

                                       ...                \
country year cause    sex              ...                 
2020    1997 homicide 1                ...                 
                      2                ...                 
                      9                ...                 

                           deaths_age_75_79_years  deaths_age_80_84_years  \
country year cause    sex                                                   
2020    1997 homicide 1                      14.0                     5.0   
                      2                       8.0                     3.0   
                      9                       0.0                     1.0   

                           deaths_age_85_89_years  deaths_age_90_94_years  \
country year cause    sex                                                   
2020    1997 homicide 1                       1.0                     0.0   
                      2                       6.0                     0.0   
                      9                       0.0                     1.0   

                           deaths_age_95_years_and_above  \
country year cause    sex                                  
2020    1997 homicide 1                              1.0   
                      2                              0.0   
                      9                              1.0   

                           deaths_age_unspecified  infant_deaths_age_0_day  \
country year cause    sex                                                    
2020    1997 homicide 1                      28.0                     30.0   
                      2                       1.0                     16.0   
                      9                       9.0                      1.0   

                           infant_deaths_age_1_6_days  \
country year cause    sex                               
2020    1997 homicide 1                           0.0   
                      2                           0.0   
                      9                           0.0   

                           infant_deaths_age_7_27_days  \
country year cause    sex                                
2020    1997 homicide 1                            0.0   
                      2    

In [163]:
icd101[icd101['Deaths1'] > 1000].query('Country == "2020" and Year == 1997').Cause.unique()

array(['A419', 'AAA', 'C159', 'C169', 'C189', 'C259', 'C349', 'C509',
       'C55', 'C61', 'C679', 'C80', 'E149', 'I10', 'I219', 'I251', 'I259',
       'I420', 'I501', 'I509', 'I619', 'I64', 'I709', 'J180', 'J189',
       'J449', 'J81', 'J969', 'J984', 'K746', 'K922', 'N189', 'R092',
       'R98', 'R99', 'X599', 'Y349'], dtype=object)

In [164]:
icd102[icd102['Deaths1'] > 1000].query('Country == "2020" and Year == 2010').Cause.unique()

array(['A419', 'AAA', 'C159', 'C169', 'C189', 'C259', 'C349', 'C509',
       'C55', 'C56', 'C61', 'C64', 'C809', 'E149', 'I10', 'I110', 'I219',
       'I259', 'I470', 'I499', 'I500', 'I501', 'I509', 'I619', 'I64',
       'I678', 'J189', 'J449', 'J690', 'J81', 'J960', 'J969', 'J984',
       'J988', 'K729', 'K746', 'N189', 'N19', 'N390', 'R99', 'V499',
       'X599', 'X700'], dtype=object)